In [1]:
import sys 

In [4]:
sys.executable 

'/Users/lotfull/miniconda3/envs/ipykernel_py3/bin/python'

In [5]:
sys.path

['',
 '/Users/lotfull/miniconda3/envs/ipykernel_py3/lib/python36.zip',
 '/Users/lotfull/miniconda3/envs/ipykernel_py3/lib/python3.6',
 '/Users/lotfull/miniconda3/envs/ipykernel_py3/lib/python3.6/lib-dynload',
 '/Users/lotfull/.local/lib/python3.6/site-packages',
 '/Users/lotfull/miniconda3/envs/ipykernel_py3/lib/python3.6/site-packages',
 '/Users/lotfull/miniconda3/envs/ipykernel_py3/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg',
 '/Users/lotfull/miniconda3/envs/ipykernel_py3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/lotfull/.ipython']

In [7]:
# coding=utf-8
import numpy
import os
import pandas as pd
import csv
import sqlite3 as lite
from bs4 import BeautifulSoup
from selenium import webdriver
from shutil import copyfile

In [10]:
def Text_from_html_object(text):
    start = text.find("<")
    finish = text.find(">")
    text = text[:start] + text[finish + 1:]
    check_for_tag = text.find('<', start)
    while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
    return text

def Text_from_html_arr(arr):
    answer_text = ''
    for object in arr:
        text = str(object)
        check_for_tag = text.find('<')
        while check_for_tag != -1:
            end_of_tag = text.find('>', check_for_tag)
            text = text[:check_for_tag] + text[end_of_tag + 1:]
            check_for_tag = text.find('<', check_for_tag)
        answer_text += text
    return answer_text

def get_page_count(soup):
    list_of_pages = soup.find_all('a', class_='page_number')
    try:
        last_page = list_of_pages[-2]
    except IndexError:
        return 1
    last_number = Text_from_html_object(str(last_page))
    return int(last_number)

def take_rus_letters_numbers(soup):
    links = []
    list_of_links = soup.find_all('a', class_='alph_menu')
    for link in list_of_links:
        pos = int(str(link).find("%D0%")) + 4
        name = str(str(link)[pos:pos+2])
        if name != "81":
            links.append(name)
    return links

def make_save(path):
    with open(path, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('name', 'type', 'group', 'definition', 'examples', 'hashtags', 'story', 'synonims'))

def add_to_save(dict, path):
    with open(path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        for word in dict:
            writer.writerow((word['name'], word['meaning'], word['type'], word['group'], 
                             word['example'], word['story'], word['hashtag'], word['synonims']))
            

    
# and then rename mycsv.out to mycsv.csv if you wish.

def get_word_from_to_path(from_i, to, filename):
    for j in range(from_i, to + 1):
    #for j in range(1, len(links)):
        path = filename + str(j) + '.csv'
        make_save(path)
        driver.get('http://teenslang.su/content/%D0%'+ rus_letters_numbers[j]) # download site to the Selenium
        html = driver.page_source # saving site source
        soup = BeautifulSoup(html, "lxml")
        page_count = get_page_count(soup)
        print('Буква алфавита № ' + str(j + 1) + ': ')
        dict = []
        for i in range(page_count):
            if i > 0:
                driver.get('http://teenslang.su/?page=' + str(i) + '&content=%D0%'+ rus_letters_numbers[j] + '*') # load sites
                html = driver.page_source # saving site source
                soup = BeautifulSoup(html, "lxml")
            print('Парсинг %d из %d' % (i + 1, page_count))
            table = soup.find('div', class_='visible-xs visible-md visible-sm visible-lg col-lg-6')
            words = table.find_all('div', class_='issueword1')

            for word_row in words:
                word_name = Text_from_html_object(str(word_row.find('span', class_='cap')))
                word_type = Text_from_html_object(str(word_row.find('span', class_='sta')))
                word_group = Text_from_html_object(str(word_row.find('span', class_='sph')))
                word_def = Text_from_html_object(str(word_row.find('span', class_='mea')))
                word_example = Text_from_html_object(str(word_row.find('span', class_='text')))
                word_story = Text_from_html_arr(word_row.find_all('div', class_='category'))
                word_hash = Text_from_html_object(str(word_row.find('span', class_='src')))
                word_syn = Text_from_html_object(str(word_row.find('span', class_='syn')))
                dict.append({
                    'num': i,
                    'name': word_name,
                    'type': word_type,
                    'group': word_group,
                    'meaning': word_def,
                    'example': word_example,
                    'story': word_story,
                    'hashtag': word_hash,
                    'synonims': word_syn
                })
        add_to_save(dict, path)
    print("!!!!!!!!!!!ВСЕ!!!!!!!!!!")
            
        
def get_rus_letters_numbers():
    driver = webdriver.PhantomJS()
    driver.get("http://teenslang.su/content/%D0%90")
    html = driver.page_source # saving site source
    soup = BeautifulSoup(html, "lxml")
    rus_letters_numbers = take_rus_letters_numbers(soup)
    return rus_letters_numbers

In [45]:
rus_letters_numbers = get_rus_letters_numbers()

In [11]:
filename = "dict"

In [ ]:
get_word_from_to_path(from_i=9, to=9, filename=filename)

In [102]:
csvs_path = "/Users/lotfull/parsing/dicts/"

In [103]:
frames = []
for i in range(0, 32):
    frames.append(pd.read_csv(csvs_path + "dict" + str(i) + ".csv"))


In [104]:
result = pd.concat(frames)

In [105]:
result.columns.values[7] = 'synonyms'

In [90]:
result.to_csv("fullDict.csv", sep=',', index=False)

In [125]:
fulldict = pd.read_csv("fullDict.csv")
a = 679
fulldict[a:a + 3]

,name,definition,type,group,examples,hashtags,story,synonyms
679,ахчи,девушка.,ж.р.,(армянский),NonNone,#Общие,NonNone,NonNone
680,Б-база,база для курения Б.,сущь.м.р.,(курение),\nБ-база\nБ-база\nБ-база\nБ-база\n\n,#Наркоманы,NonNone,NonNone
681,Б-база,Женское общежитие.,"сущ., ж.р",(студенты),Сегодня рулим на Б-Базу жрать и @бать. Лень го...,#Общие,"от исконно-русского слова ""Бл%дь"".",ЦПХ.


In [126]:
fulldict.drop_duplicates(keep='first')

,name,definition,type,group,examples,hashtags,story,synonyms
0,А сегодня я покакал,"Предложение, показывающие безразличие к тому ч...",словосочетание,"(Анонимы, тролли, хикки)",-Где можно обсудить проблемы кишечника?\n-В /p...,#Социальныесети,(происхождение второго значения) раздел был до...,NonNone
1,А то,да — выражения подтверждения.,союз и частица,NonNone,— Ты знаешь такого чела...? — А то!!!,#Общие,NonNone,да.
2,Абаля,вот это конфуз!.,NonNone,NonNone,NonNone,#Общие,NonNone,NonNone
3,Абанамат,восклицание раздражения или удивления.,междометие,NonNone,Абанамат!!! Холодильник сломался не вовремя. ...,#Общие,NonNone,"Искажение, эвфемизм матероного выражения, из п..."
4,Абапер,программист на языке ABAP/4.,"-а, м.","(айтишники\n, прогеры)","У абаперов, как правило, все проекты — это кос...",#Айтишники,"от англ. ABAPer, имеющего ровно то же значение.",АБАПер
5,Абассака,коммент к смешному креатиффу.,"-и, ж.",(каменты),NonNone,#Падонки,NonNone,"абасцака, ицпадстол, ицпадстул, обоссака, пацт..."
6,абгрейд,"АПГРЕ́ЙД\n1.Модернизация компьютера, обновлени...",Мужской род спец.,(IT),"А ты абгрейдил(апгрейдил) виндовс, до 10?",#Айтишники #Интернет,"Сфера IT, от EN слова upgrade.",АПГРЕ́ЙД.
7,абдул,Обзывание хачей.,сущ.м.р,(хачи),Абдул хач.,#Общие,NonNone,хас.
8,абдул,Мужское кавказское имя.,NonNone,NonNone,NonNone,#Общие,NonNone,NonNone
9,аберглау,Просто ляпнул - и забыл.,Существительное,(По всюду),Тоже что и Аберглау.,#Общие,город Сальск.,АбайКалаУмаласаи.


In [127]:
fulldict.to_csv("fullDict.csv", sep=',', index=False)

In [53]:
fulldict = pd.read_csv("fullDict.csv")
wordsNamesList = fulldict['name'].tolist()

In [54]:
for i, item in enumerate(wordsNamesList):
    wordsNamesList[i] = (" ".join(item.split())).capitalize()
wordsNamesList[:3]

['А сегодня я покакал', 'А то', 'Абаля']

In [56]:
WordsNamesSeries = pd.Series(wordsNamesList).drop_duplicates(keep='first')
FinalWordsNamesList = WordsNamesSeries.tolist()
FinalWordsNamesList

['А сегодня я покакал',
 'А то',
 'Абаля',
 'Абанамат',
 'Абапер',
 'Абассака',
 'Абгрейд',
 'Абдул',
 'Аберглау',
 'Абзац',
 'Абздольц',
 'Абибас',
 'Абик',
 'Абила',
 'Абилка',
 'Абитура',
 'Абитуха',
 'Абориген',
 'Абрек',
 'Абсурд',
 'Абуба',
 'Абузить',
 'Абуминог',
 'Абуэма',
 'Абыр',
 'Абьюзер',
 'Ав',
 'Ава',
 'Авапёр',
 'Авас',
 'Аватар',
 'Аватара',
 'Аватарка',
 'Авдотья',
 'Авдюха',
 'Аве',
 'Аве хоггарт, аве сотона',
 'Авео',
 'Авепешник',
 'Авер',
 'Авик',
 'Авишка',
 'Авишник',
 'Авоська',
 'Авп',
 'Австралопитек',
 'Автаз',
 'Автар',
 'Автик',
 'Авто-моб',
 'Автобан',
 'Автобат',
 'Автобус',
 'Автогад',
 'Автоглюк',
 'Автолюбители',
 'Автомат',
 'Автопати',
 'Автопилотка',
 'Автор вопроса',
 'Автор вопроса фиктивный',
 'Автор словаря молодежный сленг',
 'Авторитетный',
 'Авторская версия ответа',
 'Авторская группа',
 'Авторская песня',
 'Авторский моб',
 'Авца',
 'Авчик',
 'Ага',
 'Агал',
 'Аганимус',
 'Агарка',
 'Агась',
 'Агенты',
 'Аглицкий',
 'Агностик',
 'Агнст',


In [20]:
print(wordsNamesList)

['А сегодня я покакал', 'А то', 'Абаля', 'Абанамат', 'Абапер', 'Абассака', 'абгрейд', 'абдул', 'абдул', 'аберглау', 'Абзац', 'Абздольц', 'Абибас', 'Абик', 'Абила', 'абилка', 'Абитура', 'Абитуха', 'Абориген', 'Абрек', 'абсурд', 'Абуба', 'абузить', 'абуминог', 'Абуэма', 'абыр', 'абьюзер', 'АВ', 'Ава', 'Ава', 'ава', 'Авапёр', 'Авас', 'АВАС', 'Аватар', 'Аватара', 'Аватарка', 'Авдотья', 'Авдюха', 'Аве', 'Аве Хоггарт, аве сотона', 'Авео', 'Авепешник', 'Авер', 'Авик', 'Авишка', 'Авишник', 'Авоська', 'авп', 'Австралопитек', 'Автаз', 'Автар', 'Автик', 'Автик', 'Авто-моб', 'Автобан', 'автобат', 'Автобус', 'Автогад', 'Автоглюк', 'Автолюбители', 'Автомат', 'Автопати', 'Автопилотка', 'Автор вопроса', 'Автор вопроса', 'Автор вопроса фиктивный', 'автор словаря молодежный сленг', 'Авторитетный', 'Авторская версия ответа', 'Авторская группа', 'Авторская песня', 'Авторский моб', 'авца', 'Авчик', 'Ага', 'агал', 'аганимус', 'агарка', 'Агась', 'Агенты', 'Аглицкий', 'агностик', 'агнст', 'Агр', 'Агрессив инл

In [33]:
wordsNames = fulldict['name']

In [35]:
def capitalize_and_strip(x):
    return x.capitalize().strip()

In [37]:
wordsNames.apply(capitalize_and_strip)
wordsNames.tolist()

['А сегодня я покакал',
 'А то',
 'Абаля',
 'Абанамат',
 'Абапер',
 'Абассака',
 'абгрейд',
 'абдул',
 'абдул',
 'аберглау',
 'Абзац',
 'Абздольц',
 'Абибас',
 'Абик',
 'Абила',
 'абилка',
 'Абитура',
 'Абитуха',
 'Абориген',
 'Абрек',
 'абсурд',
 'Абуба',
 'абузить',
 'абуминог',
 'Абуэма',
 'абыр',
 'абьюзер',
 'АВ',
 'Ава',
 'Ава',
 'ава',
 'Авапёр',
 'Авас',
 'АВАС',
 'Аватар',
 'Аватара',
 'Аватарка',
 'Авдотья',
 'Авдюха',
 'Аве',
 'Аве Хоггарт, аве сотона',
 'Авео',
 'Авепешник',
 'Авер',
 'Авик',
 'Авишка',
 'Авишник',
 'Авоська',
 'авп',
 'Австралопитек',
 'Автаз',
 'Автар',
 'Автик',
 'Автик',
 'Авто-моб',
 'Автобан',
 'автобат',
 'Автобус',
 'Автогад',
 'Автоглюк',
 'Автолюбители',
 'Автомат',
 'Автопати',
 'Автопилотка',
 'Автор вопроса',
 'Автор вопроса',
 'Автор вопроса фиктивный',
 'автор словаря молодежный сленг',
 'Авторитетный',
 'Авторская версия ответа',
 'Авторская группа',
 'Авторская песня',
 'Авторский моб',
 'авца',
 'Авчик',
 'Ага',
 'агал',
 'аганимус',
 'ага

In [13]:
myDic={'b': 2, 'a': 3, 'c': 1}
def print_dict_sorted_by_value(dictionary):
    for (key, value) in sorted(dictionary.items(), key=lambda x: x[1], reverse=True):
        print(key, ": ", value, sep='')

a: 3
b: 2
c: 1
